za podatke https://github.com/WarrierRajeev/UFC-Predictions <br>
elo https://en.wikipedia.org/wiki/Elo_rating_system <br>
postojeći elo https://github.com/infinitelytight/mma-elo <br>
scraping https://realpython.com/beautiful-soup-web-scraper-python/


- mozes scrapenja opisati
- inputaj si podatke
- nacrtaj neke grafove
- objasni elo
- primjeni elo neki svoj/napravljen
- objasni sto je submission sto udarac
- kritika elo sistema 
- preporuka novog
- chessboxing

In [1]:
import pandas as pd
import numpy as np
import operator

In [2]:
fighter_details = pd.read_csv("fighter_details.csv", encoding='utf-8')[['fighter_name', 'Weight']]

In [3]:
fighter_details = fighter_details.set_index('fighter_name')
fighter_details.insert(1, "Elo rating", 1500)

In [4]:
fighter_details.head()

,Weight,Elo rating
fighter_name,,
Ricardo Abreu,185 lbs.,1500
Papy Abedi,185 lbs.,1500
Klidson Abreu,205 lbs.,1500
Shamil Abdurakhimov,235 lbs.,1500
Daniel Acacio,180 lbs.,1500


In [5]:
total_fight_data = pd.read_csv("total_fight_data.csv", delimiter=";", 
                               encoding='utf-8')[["R_fighter", "B_fighter", "win_by", 
                                                  "date", "Fight_type", "Winner"]]

In [6]:
total_fight_data.head()

,R_fighter,B_fighter,win_by,date,Fight_type,Winner
0,Sean O'Malley,Thomas Almeida,KO/TKO,"March 27, 2021",Bantamweight Bout,Sean O'Malley
1,Jared Gooden,Abubakar Nurmagomedov,Decision - Unanimous,"March 27, 2021",Welterweight Bout,Abubakar Nurmagomedov
2,Alonzo Menifield,Fabio Cherant,Submission,"March 27, 2021",Light Heavyweight Bout,Alonzo Menifield
3,Stipe Miocic,Francis Ngannou,KO/TKO,"March 27, 2021",UFC Heavyweight Title Bout,Francis Ngannou
4,Shane Young,Omar Morales,Decision - Unanimous,"March 27, 2021",Featherweight Bout,Omar Morales


In [7]:
total_fight_data.head()

,R_fighter,B_fighter,win_by,date,Fight_type,Winner
0,Sean O'Malley,Thomas Almeida,KO/TKO,"March 27, 2021",Bantamweight Bout,Sean O'Malley
1,Jared Gooden,Abubakar Nurmagomedov,Decision - Unanimous,"March 27, 2021",Welterweight Bout,Abubakar Nurmagomedov
2,Alonzo Menifield,Fabio Cherant,Submission,"March 27, 2021",Light Heavyweight Bout,Alonzo Menifield
3,Stipe Miocic,Francis Ngannou,KO/TKO,"March 27, 2021",UFC Heavyweight Title Bout,Francis Ngannou
4,Shane Young,Omar Morales,Decision - Unanimous,"March 27, 2021",Featherweight Bout,Omar Morales


In [8]:
total_fight_data["date"] = pd.to_datetime(total_fight_data["date"], format='%B %d, %Y')

In [9]:
total_fight_data = total_fight_data.sort_values(by = ["date"])

In [10]:
most_wins = total_fight_data[["Winner"]]
most_wins.insert(1,"Count",0)
most_wins.groupby("Winner").count().sort_values("Count", ascending = False).head(10)

,Count
Winner,
Donald Cerrone,23
Demian Maia,22
Jim Miller,21
Jon Jones,20
Michael Bisping,20
Georges St-Pierre,20
Rafael Dos Anjos,19
Andrei Arlovski,19
Dustin Poirier,19


In [11]:
class Elo:
    _win = 1
    _draw = 0.5
    _lose = 0
    
    def __init__(self, K = 32, beta = 400, title_scale = 1):
        self._K = K
        self._beta = beta
        self._title_scale = title_scale
    
    def _real_result(self, red, blue, winner, is_title):
        win = self._win
        
        if is_title:
            win = self._title_scale * win
            
        if (red == winner):
            return  win, self._lose
        
        elif (blue == winner):
            return self._lose, win
        
        else:
            return self._draw, self._draw
        
    def _expectation(self, a, b):
        return 1 / (1 + 10**((b - a) / self._beta))
        
    def _expected_result(self, red_elo, blue_elo):
        return self._expectation(red_elo, blue_elo), self._expectation(blue_elo, red_elo)
    
    def _new_elo_calc(self, a, res, expectation):
        return a + self._K * ( res - expectation )
    
    def new_elo(self, red, blue, winner, is_title = False):
        real_red, real_blue = self._real_result(red.name, blue.name, winner, is_title)
        
        expectation_red, expectation_blue = self._expected_result(red["Elo rating"], blue["Elo rating"])
        
        red = red.replace(red["Elo rating"], self._new_elo_calc(red["Elo rating"], real_red, expectation_red))
        blue = blue.replace(blue["Elo rating"], self._new_elo_calc(blue["Elo rating"], real_blue, expectation_blue))
        
        return red, blue

In [12]:
def test_elo(fighter_details, K, beta, title_scale = 1):
    elo = Elo(K, beta, title_scale)
    for _, row in total_fight_data.iterrows():
        red_fighter = row["R_fighter"]
        blue_fighter = row["B_fighter"]
        winner = row["Winner"]
        red_fighter_data = fighter_details.loc[red_fighter]
        blue_fighter_data = fighter_details.loc[blue_fighter]
        fighter_details.loc[red_fighter], fighter_details.loc[blue_fighter] = elo.new_elo(red_fighter_data, 
                                                                                          blue_fighter_data, 
                                                                                          winner, "Title" in row["Fight_type"])
    print(fighter_details.sort_values(by = ["Elo rating"], ascending = False).head(10),"\n\n")
    
    print(fighter_details.sort_values(by = ["Elo rating"], ascending = True).head(10))

In [13]:
fdets_copy = fighter_details.copy()
test_elo(fdets_copy, 32, 400)

                       Weight   Elo rating
fighter_name                              
Jon Jones            205 lbs.  1750.865399
Georges St-Pierre    185 lbs.  1746.811470
Kamaru Usman         170 lbs.  1698.754105
Khabib Nurmagomedov  155 lbs.  1692.617958
Dustin Poirier       155 lbs.  1679.520922
Amanda Nunes         145 lbs.  1675.402825
Demetrious Johnson   125 lbs.  1667.645159
Max Holloway         145 lbs.  1660.156839
Charles Oliveira     155 lbs.  1657.985935
Tony Ferguson        155 lbs.  1657.838133 


                     Weight   Elo rating
fighter_name                            
Joshua Burkman     170 lbs.  1420.046223
Leonard Garcia     145 lbs.  1426.535658
Naoyuki Kotani     155 lbs.  1428.211253
Kailin Curran      115 lbs.  1428.293244
Michihiro Omigawa  145 lbs.  1429.981257
John Alessio       155 lbs.  1431.943023
Phillipe Nover     145 lbs.  1432.817733
Elvis Sinosic      205 lbs.  1434.620618
Steve Cantwell     185 lbs.  1437.691428
Vinicius Moreira   205 lbs.  1

In [14]:
fdets_copy = fighter_details.copy()
test_elo(fdets_copy, 72, 200)

                         Weight   Elo rating
fighter_name                                
Georges St-Pierre      185 lbs.  1894.390090
Kamaru Usman           170 lbs.  1891.069558
Jon Jones              205 lbs.  1888.510240
Khabib Nurmagomedov    155 lbs.  1870.053007
Francis Ngannou        250 lbs.  1839.864866
Alexander Volkanovski  145 lbs.  1832.897031
Dustin Poirier         155 lbs.  1821.886374
Stipe Miocic           240 lbs.  1820.010622
Jan Blachowicz         205 lbs.  1814.779293
Henry Cejudo           135 lbs.  1812.860372 


                    Weight   Elo rating
fighter_name                           
Leonard Garcia    145 lbs.  1365.481251
Vinicius Moreira  205 lbs.  1373.963174
Steve Cantwell    185 lbs.  1375.232473
Jodie Esquibel    115 lbs.  1376.960124
Josh Grispi       145 lbs.  1379.492097
Terrion Ware      135 lbs.  1380.681831
Andy Ogle         145 lbs.  1380.973520
Edgar Garcia      170 lbs.  1381.112259
Naoyuki Kotani    155 lbs.  1381.310814
Kailin Curran    

In [15]:
fdets_copy = fighter_details.copy()
test_elo(fdets_copy, 32, 400, 2)

                       Weight   Elo rating
fighter_name                              
Jon Jones            205 lbs.  2128.954708
Georges St-Pierre    185 lbs.  2117.142486
Demetrious Johnson   125 lbs.  1983.714839
Amanda Nunes         145 lbs.  1939.057166
Stipe Miocic         240 lbs.  1850.692782
Kamaru Usman         170 lbs.  1847.881425
Daniel Cormier       235 lbs.  1835.932047
Khabib Nurmagomedov  155 lbs.  1820.563671
Max Holloway         145 lbs.  1808.625394
Anderson Silva       185 lbs.  1805.602680 


                     Weight   Elo rating
fighter_name                            
Joshua Burkman     170 lbs.  1420.566537
Leonard Garcia     145 lbs.  1426.543987
Kailin Curran      115 lbs.  1428.293244
Naoyuki Kotani     155 lbs.  1429.415980
Michihiro Omigawa  145 lbs.  1429.987285
Phillipe Nover     145 lbs.  1436.562319
John Alessio       155 lbs.  1437.070439
Steve Cantwell     185 lbs.  1437.696168
Vinicius Moreira   205 lbs.  1437.851706
Jodie Esquibel     115 lbs.  1

In [16]:
fdets_copy = fighter_details.copy()
test_elo(fdets_copy, 72, 200, 2)

                        Weight   Elo rating
fighter_name                               
Jon Jones             205 lbs.  2793.036497
Georges St-Pierre     185 lbs.  2756.831267
Demetrious Johnson    125 lbs.  2440.694697
Amanda Nunes          145 lbs.  2391.426949
Stipe Miocic          240 lbs.  2282.366679
Daniel Cormier        235 lbs.  2224.933272
Kamaru Usman          170 lbs.  2213.001661
Khabib Nurmagomedov   155 lbs.  2166.230410
Henry Cejudo          135 lbs.  2130.602800
Valentina Shevchenko  125 lbs.  2076.952879 


                    Weight   Elo rating
fighter_name                           
Leonard Garcia    145 lbs.  1366.192047
Vinicius Moreira  205 lbs.  1375.204540
Steve Cantwell    185 lbs.  1375.706987
Josh Grispi       145 lbs.  1379.492289
Jodie Esquibel    115 lbs.  1379.520384
Terrion Ware      135 lbs.  1381.107585
Edgar Garcia      170 lbs.  1381.219983
Kailin Curran     115 lbs.  1384.725139
Andy Ogle         145 lbs.  1384.936485
Naoyuki Kotani    155 lbs.  1

In [17]:
fd_copy = fighter_details.copy()
fd_copy["Weight"] = fd_copy["Weight"].str.replace(" lbs.","").astype(float)
fd_copy.head()

,Weight,Elo rating
fighter_name,,
Ricardo Abreu,185.0,1500
Papy Abedi,185.0,1500
Klidson Abreu,205.0,1500
Shamil Abdurakhimov,235.0,1500
Daniel Acacio,180.0,1500


### TODOS:
- PEP neki prati
- neki grafovi
- malo izvrti još ovo (vidi sto bude najsmislenije)

In [18]:
#install usupported libraries
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install requests

In [19]:
import requests
from bs4 import BeautifulSoup

In [20]:
def web_scrape(url, identifier, container):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all(container, class_ = identifier)
    for result in results:
        print(result.text.strip())

In [21]:
web_scrape("https://www.unixtimestamp.com/", "value epoch", "div")

1617439457


In [22]:
web_scrape("http://www.pmf.unizg.hr/math", ["news_article_title", "news_short_title"], "div")

Natječaji za stipendije francuske vlade 2021/2022
Potvrde o diplomiranju
BEST Zagreb - Obavijest o primanju novih članova i uvodnom sastanku
Udruga CISEx, u suradnji s udrugom CroAI, pokreće hrvatsku AI ligu
Natječaj za dodjelu Rektorove nagrade za ak. god. 2020./21.
Važna obavijest studentima (28.01.)
PMF priprema prvi studij u Hrvatskoj iz biomedicinske matematike
